<a href="https://colab.research.google.com/github/AlbaneMiron/drone-simulation/blob/master/drone_nofly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import geopandas as gpd
from shapely.geometry import Point, LineString
from queue import PriorityQueue
from heapq import heappop, heappush

In [3]:
pip install pyvisgraph

  Preparing metadata (setup.py) ... done
  Created wheel for pyvisgraph: filename=pyvisgraph-0.2.1-py3-none-any.whl size=12606 sha256=ffc161ef100391b1bb5ce35ba2d950edc77b77b279ee79be06ac735e2b2521e5
  Stored in directory: /root/.cache/pip/wheels/2d/f6/fa/7550eb60ebfc841ee3fbd5adadbeedb2586f0095d7f2f2f465
Successfully built pyvisgraph


In [4]:
import geopandas as gpd
import pyvisgraph as vg
import math

# Fonction pour charger les polygones (et gérer les multipolygones) depuis un fichier GeoJSON
def load_polygons_from_geojson(filepath):
    gdf = gpd.read_file(filepath)  # Charger le fichier GeoJSON avec GeoPandas
    polygons = []

    for geom in gdf.geometry:
        if geom.geom_type == 'Polygon':  # Si la géométrie est un polygone
            coordinates = list(geom.exterior.coords)  # Contour externe
            polygon = [vg.Point(coord[0], coord[1]) for coord in coordinates]
            polygons.append(polygon)
        elif geom.geom_type == 'MultiPolygon':  # Si la géométrie est un multipolygone
            for poly in geom.geoms:  # Extraire chaque polygone du multipolygone
                coordinates = list(poly.exterior.coords)  # Contour externe
                polygon = [vg.Point(coord[0], coord[1]) for coord in coordinates]
                polygons.append(polygon)
        else:
            print(f"Type de géométrie non pris en charge : {geom.geom_type}")

    return polygons

# Fonction pour calculer la distance entre deux points GPS (utilise la distance de Haversine)
def haversine_distance(point1, point2):
    R = 6371  # Rayon de la Terre en kilomètres
    lat1, lon1 = math.radians(point1[1]), math.radians(point1[0])
    lat2, lon2 = math.radians(point2[1]), math.radians(point2[0])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

In [14]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon

# Définir une fonction pour vérifier si une géométrie est en France métropolitaine ou en Corse
def is_in_metropolitan_france_or_corsica(geometry):
    # Bounding box approximative pour la France métropolitaine et la Corse
    france_bbox = {
        "min_x": -5.00000, "max_x": 10.00000,
        "min_y": 41.00000, "max_y": 51.50000
    }

    # Vérification si la géométrie est totalement ou partiellement dans la boîte
    if geometry.bounds[0] > france_bbox["max_x"] or geometry.bounds[2] < france_bbox["min_x"]:
        return False
    if geometry.bounds[1] > france_bbox["max_y"] or geometry.bounds[3] < france_bbox["min_y"]:
        return False

    # Vérification géographique précise si la géométrie croise ou est dans les limites
    bbox_polygon = Polygon([
        (france_bbox["min_x"], france_bbox["min_y"]),
        (france_bbox["max_x"], france_bbox["min_y"]),
        (france_bbox["max_x"], france_bbox["max_y"]),
        (france_bbox["min_x"], france_bbox["max_y"]),
        (france_bbox["min_x"], france_bbox["min_y"])
    ])
    return geometry.intersects(bbox_polygon)

# Charger le fichier GeoJSON
def filter_geojson(input_geojson_path, output_geojson_path):
    # Charger les données GeoJSON
    gdf = gpd.read_file(input_geojson_path)

    # Filtrer les géométries en France métropolitaine ou Corse
    filtered_gdf = gdf[gdf.geometry.apply(is_in_metropolitan_france_or_corsica)]

    # Sauvegarder les données filtrées dans un nouveau fichier GeoJSON
    filtered_gdf.to_file(output_geojson_path, driver="GeoJSON")

# Exemple d'utilisation
input_geojson = "Arrete_ZICAD_10-2024.geojson"
output_geojson = "Arrete_ZICAD_10-2024_filtre.geojson"
filter_geojson(input_geojson, output_geojson)

In [15]:
# Charger les polygones depuis le fichier GeoJSON
geojson_file = "Arrete_ZICAD_10-2024_filtre.geojson"
polygons = load_polygons_from_geojson(geojson_file)

# Construire le graphe de visibilité
graph = vg.VisGraph()
graph.build(polygons)


100%|██████████| 507/507 [13:04<00:00,  1.55s/it]


In [13]:
import pyvisgraph as vg
import folium
from shapely.geometry import LineString

# Étape 3 : Initialiser une carte Folium
m = folium.Map(location=[48.856613, 2.352222], zoom_start=13)

# Étape 4 : Ajouter les nœuds et arêtes du graphe à la carte
for edge in graph.visgraph.edges:  # Pas de parenthèses ici
    # Convertir les points de l'arête en coordonnées GPS
    p1 = edge.p1
    p2 = edge.p2

    # Ajouter une ligne à la carte pour représenter l'arête
    folium.PolyLine([(p1.y, p1.x), (p2.y, p2.x)], color='blue', weight=2).add_to(m)

# Ajouter les sommets (points) à la carte
for vertex in graph.visgraph.graph:
    folium.CircleMarker(location=(vertex.y, vertex.x), radius=3, color='red', fill=True).add_to(m)

# Étape 5 : Sauvegarder ou afficher la carte
m.save("graphe_de_visibilite.html")
m



KeyboardInterrupt: 

In [16]:
graph.save('graph.pk1')
# graph = VisGraph()
# graph.load('graph.pk1')

In [21]:
# Points GPS pour lesquels on veut calculer la distance
start = vg.Point(2.339064, 48.834706)  # Paris, 92 boulevard Arago
end = vg.Point(2.341189, 48.832675)   # Paris, 67 rue de la Santé

# Trouver le chemin le plus court entre deux points dans le graphe
shortest_path = graph.shortest_path(start, end)

# Calculer la distance totale du chemin
path_distance = sum(
    haversine_distance((shortest_path[i].x, shortest_path[i].y), (shortest_path[i+1].x, shortest_path[i+1].y))
    for i in range(len(shortest_path) - 1)
)

# Afficher les coordonnées GPS du chemin le plus court avec 6 chiffres après la virgule
shortest_path_coords = [(round(point.y, 6), round(point.x, 6)) for point in shortest_path]
print("Chemin le plus court (coordonnées) :", shortest_path_coords)
print("Distance totale : {:.4f} km".format(path_distance))

Chemin le plus court (coordonnées) : [(48.834706, 2.339064), (48.834444, 2.341389), (48.832675, 2.341189)]
Distance totale : 0.3699 km


In [27]:
import folium

def plot_route_on_map(route, map_center, zoom_start=15):
    """
    Plot a route on a map using Folium.

    Parameters:
        route (list of tuple): The route as a list of (latitude, longitude) coordinates.
        map_center (tuple): The center of the map as (latitude, longitude).
        zoom_start (int): Initial zoom level of the map.
    """
    # Create a Folium map
    m = folium.Map(location=map_center, zoom_start=zoom_start)

    # Add the route to the map as a polyline
    folium.PolyLine(route, color="blue", weight=4, opacity=0.8).add_to(m)

    folium.Polygon(
    locations=[[48.8344444444444, 2.33805555555552],
     [48.8344444444444, 2.34138888888892],
     [48.8330555555556, 2.3411111111111],
     [48.8336111111111, 2.33805555555552],
     [48.8344444444444, 2.33805555555552]],
    color="red",
    weight=0.1,
    fill_color="red",
    fill_opacity=0.5,
    fill=True).add_to(m)

    # Add start and end markers
    folium.Marker(location=route[0], icon=folium.Icon(color="green"), popup="Start").add_to(m)
    folium.Marker(location=route[-1], icon=folium.Icon(color="red"), popup="End").add_to(m)

    return m

def main():
    # Example route generated by the algorithm
    route = [(48.834706, 2.339064), (48.834444, 2.341389), (48.832675, 2.341189)]

    # Define the center of the map (midpoint between start and end)
    map_center = ((route[0][0] + route[-1][0]) / 2, (route[0][1] + route[-1][1]) / 2)

    # Generate the map
    m = plot_route_on_map(route, map_center)

    # Save and display the map
    m.save("route_map.html")
    print("Map has been saved to 'route_map.html'.")

if __name__ == "__main__":
    main()

Map has been saved to 'route_map.html'.
